<a href="https://colab.research.google.com/github/hogo56/BertQA/blob/master/BERT_for_Humans_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## --Main System Config--
<Details>Put any global system configuration variables here</Details>

EnableAllCode - There are code blocks here that should not be run with "Run All". By default EnableAllCode will set to False and those blocks will be excluded. If you want to run them for some reason set EnableAllCode True.<p>
The current setup code requires manual authentication to Google Drive and Kaggle so it cannot be run unattended.

In [0]:
import os, sys

In [0]:
EnableAllCode = True

In [0]:
EnableAllCode = False

## -- Setup --

If running for first time I would suggest Edit -> Clear All Outputs before starting

### Machine Setup

--- Google Drive
<Details>There are several ways to provide access to your Google Drive from Colab.<br>
I am not sure if this is the best. This mounts your Drive into the machine.<br>
I expect there will be a folder in the Drive that we all share.</Details>

In [0]:
## File link to Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)   # true to reread drive
# Create a shorter shared directory name than one with a space
! ln -s '/content/gdrive/My Drive/bertqa' /content/bertqa

In [0]:
if EnableAllCode:
    ## Flush and unmount Google Drive
    # You probablyu won't do this but if you want to at some point click the play button
    drive.flush_and_unmount()

### SSH Setup
<Details> Shell access is not technically required but if you are like me and being able to look under the hood brings you comfort you can step throuth this.<br>
Thanks to Imad El Hanafi (https://imadelhanafi.com) for showing me how to do this.</Details>

In [0]:
# 1 - setup ssh/user 

#Generate a random root password
import random, string
sshpass = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(30))

#Setup sshd
! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null

#Set root password
! echo root:$sshpass | chpasswd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config

print("username: root")
print("password: ", sshpass)

#Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')

In [0]:
# 2 - Download Ngrok
! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip
! rm ngrok-stable-linux-amd64.zip


I think you will need to create a free account at https://ngrok.com/ for the SSH tunnel to work.

In [0]:
# 3 - setup Ngrok - authtoken

#Ask token
print("Get your authtoken from https://dashboard.ngrok.com/auth")
import getpass
authtoken = getpass.getpass()

#Create tunnel
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')

Congratulations, you are ready to go. 
On Ngrok dashboard https://dashboard.ngrok.com/status you'll find the tcp address and the port you can connect to with your favorite ssh client. eg...

```
ssh root@0.tcp.ngrok.io -p [ngrok_port]
> then enter the username and password shown from the codeblock above

root@hostname:~ PS1=':\w\$ '
:~# apt install vim
:~# vim .bashrc        (if you want to change prompt PS1 there; also add cd/content to end)
:~# cd /content
:/content# ls -lh 
```

In [0]:
if EnableAllCode:
    # When done, kill Ngrok
    !kill $(ps aux | grep './ngrok' | awk '{print $2}')

### Kaggle API
<Details>You will need a token to link the Colab instance to the API of your Kaggle account to get data, etc.<br>
Go to: https://www.kaggle.com/yourID/account and click on the "Create New API Token: button to get a file named kattle.json and save it somewhere on your local drive.</Details>
<Details><Summary>More...</Summary>I am choosing to upload the kaggle.json file from local hard drive each time. Alternately, we could put it in a private location on your gdrive (not the shared folder.)<br>
If you want to use Drive, edit the location in the environ var so python knows where to find your kaggle.json file. By using a file upload however, the same code will work for everyone sharing this notebook</Details>

In [0]:
## Link to Kaggle
# I am getting a sporatic error "Cannot read property '_uploadFiles'"; just retry and it works
import os
from google.colab import files

!rm /content/kaggle.json
print('Upload kaggle.json.')
# You will need a kaggle auth file from your account
uploaded = files.upload()
!chmod 600 kaggle.json
os.environ['KAGGLE_CONFIG_DIR'] = "/content/"
!ls -l /content/kaggle.json

import kaggle

Here is where you download the competition dataset. Only do this if you need it for your current work as it is 5GB (zipped)

In [0]:
!mkdir -p /content/data
!mkdir -p /content/bert_output       # Maybe output to Google Drive for durability
# Get Competition Data (5GB zipped)
!kaggle competitions list
!kaggle competitions download -c tensorflow2-question-answering -p /content/data

### Library Setup

In [0]:
## Fix Nvidia            (make sure your Runtime type is python3 + GPU)

""" ToDo - Get Latest Nvidia Drivers """
if EnableAllCode:
    # https://developer.nvidia.com/cudnn       (need to join developer program to get latest)

    # Extracts the cuDNN files from Drive folder directly to the VM CUDA folders
    !tar -xzvf bertqa/nvidia/cuDNN/cudnn-10.0-linux-x64-v7.5.0.56.tgz -C /usr/local/
    !chmod a+r /usr/local/cuda/include/cudnn.h

    # Now we check the version we already installed. Can comment this line on future runs
    !cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2



In [0]:
## Setup local lib
! mkdir /content/lib
sys.path.append('/content/lib')
! cp -a /content/bertqa/lib/* lib/

In [0]:
# get BERT   (https://www.kaggle.com/coolcoder22/colab-to-fine-tune-bert)
! git clone https://github.com/google-research/bert.git

# get pretrained models
! wget https://storage.googleapis.com/bert_models/2018_10_18/cased_L-12_H-768_A-12.zip
! unzip cased_L-12_H-768_A-12.zip
! rm cased_L-12_H-768_A-12.zip
! mv bert lib

Are these the correct bert models we are supposed to be using?<br>
Maybe also look at https://github.com/tensorflow/models/tree/master/official/nlp/bert or,<br>
https://github.com/google-research/language/tree/master/language/question_answering/bert_joint

<Details><Summary>BERT Notes</Summary>
baseline_w_bert_translated_to_tf2_0 (next code block) comes from /dimitreoliveira with this warning:<br>
This baseline uses code that was migrated from TF1.x. Be aware that it contains use of tf.compat.v1, which is not permitted to be eligible for TF2.0 prizes in this competition. It is intended to be used as a starting point, but we're excited to see how much better you can do using TF2.0!<br>
https://www.kaggle.com/dimitreoliveira/using-tf-2-0-w-bert-on-nq-translated-to-tf2-0</Details>

In [0]:
## Load Libraries

%tensorflow_version 2.x 
import tensorflow
print(tensorflow.__version__)

import numpy as np
import pandas as pd
import tensorflow as tf

# import tf2_0_baseline_w_bert as tf2baseline # old script
import tf2_0_baseline_w_bert_translated_to_tf2_0 as tf2baseline # Oliviera's script

import bert.modeling as modeling
import bert.optimization as optimization
import bert.tokenization as tokenization

import json
import absl
from zipfile import ZipFile

## --Explore Environment--


In [0]:
if EnableAllCode:
    print('===== Python =====')
    print('Version: ', sys.version)
    ! pip -V
    print('===== CPU Info =====')
    !cat /proc/cpuinfo
    print('\n===== MEM Info =====')
    !cat /proc/meminfo
    print('\n===== printenv =====')
    ! printenv
    from tensorflow.python.client import device_lib
    print('\n===== List Local Devices =====')
    print(device_lib.list_local_devices())
    print()
    print("===== Dir of /:", *(os.listdir('/')), sep='\n')
    print("\n===== cwd: ", os.getcwd())                       # will be /content
    print('\n===== Files in', os.getcwd())
    for dirname, _, filenames in os.walk(os.getcwd()):
        for filename in filenames:
            print(os.path.join(dirname, filename))

In [0]:
if EnableAllCode:
    print('\n===== NVIDIA Info =====')
    !nvidia-smi
    print()
    !/usr/local/cuda/bin/nvcc --version

## --Misc Notes--

### File Transfer to Local Machine

In [0]:
## To upload files to Colab
from google.colab import files
uploaded = files.upload()
print(uploaded)

In [0]:
## To download files from Colab
from google.colab import files
model.save('trained_model.h5')          # but probably better to save directly to Drive
files.download('trained_model.h5')

### File Transfer from Web

In [0]:
from urllib.request import urlretrieve

def download(url, file):
    if not os.path.isfile(file):
        print("Downloading file... " + file + " ...")
        urlretrieve(url,file)
        print("File downloaded")

download('Url of the file','Name of the file to be saved')
print("All the files are downloaded")
#If the downloaded file is a zip file than you can use below function to unzip it.
def uncompress_features_labels(file):
    if(os.path.isdir('data')):
        print('Data extracted')
    else:
        with ZipFile(dir) as zipf:
            zipf.extractall('data')

In [0]:
! printenv

### Prevent Disconnects
Colab periodically disconnects the browser.<br>
You have to save model checkpoints to Google Drive so you don't lose wor<br>
See: https://mc.ai/google-colab-drive-as-persistent-storage-for-long-training-runs/<br>
Something to try...<br>
Ctrl+Shift+i in browser and in console run this code...
```
function KeepAlive(){
    console.log("Maintaining Connection");
    document.querySelector("colab-toolbar-button#connect").click()
}
setInterval(KeepAlive,60000);
```
There have been reports of people having their GPU privileges suspended for letting processes run for over 12 hours. It seems that they may penalize you rather than just cutting you off.